In [1]:
import tensorflow as tf
import numpy as np
from model import EXTRACTOR, DNN
import os, re, glob
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import skimage
import skimage.io

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
video_path_train = '/data/put_data/cclin/ntu/dlcv2018/hw5/HW5_data/TrimmedVideos/video/train'
label_file_train = '/data/put_data/cclin/ntu/dlcv2018/hw5/HW5_data/TrimmedVideos/label/gt_train.csv'
feature_path_train = '/data/put_data/cclin/ntu/dlcv2018/hw5/cnn_features_train.npy'

video_path_valid = '/data/put_data/cclin/ntu/dlcv2018/hw5/HW5_data/TrimmedVideos/video/valid'
label_file_valid = '/data/put_data/cclin/ntu/dlcv2018/hw5/HW5_data/TrimmedVideos/label/gt_valid.csv'
feature_path_valid = '/data/put_data/cclin/ntu/dlcv2018/hw5/cnn_features_valid.npy'

### Extract CNN-based features

#### (1) extract training features

In [3]:
tf.reset_default_graph()

with tf.Session() as sess:
    net = EXTRACTOR(sess)
    net.build_vgg16()
    res = net.extract(video_path=video_path_train,
                      label_file=label_file_train)

vgg16.npy loaded
RGB to BGR
build model started


In [4]:
## debug
print(type(res))
print(len(res))
print(type(res[0]))
print(res[0].shape)
print(res[0])

<class 'list'>
3236
<class 'numpy.ndarray'>
(4, 40960)
[[0.        0.        0.        ... 0.        0.        5.4289293]
 [0.        0.        0.        ... 0.        4.7238235 0.       ]
 [0.        0.        0.        ... 0.        4.732338  5.7481074]
 [0.        0.        0.        ... 0.        0.        1.9387643]]


In [5]:
np.save(feature_path_train, res)

#### (2) extract validation features

In [6]:
tf.reset_default_graph()

with tf.Session() as sess:
    net = EXTRACTOR(sess)
    net.build_vgg16()
    res = net.extract(video_path=video_path_valid,
                      label_file=label_file_valid)

vgg16.npy loaded
RGB to BGR
build model started


In [7]:
np.save(feature_path_valid, res)

### Train the discriminative model

In [3]:
dir_name = 'cnn_dnn_2048_512_11_bs32_lr5e5_ep50'
bsize = 32
learning_rate = 5e-5
num_epoch = 50

In [4]:
tf.reset_default_graph()

with tf.Session() as sess:
    net = DNN(sess,
              model_name=dir_name)
    net.build_model()
    results = net.train(feature_path_train=feature_path_train,
                        bsize=bsize,
                        learning_rate=learning_rate,
                        num_epoch=num_epoch)

Epoch: 1, train loss: 2.016451, valid loss: 1.770868, train accuracy: 0.316483, valid accuracy: 0.411397
Epoch: 2, train loss: 1.016238, valid loss: 1.646856, train accuracy: 0.855392, valid accuracy: 0.423529
stopping_step = 0
Epoch: 3, train loss: 0.559776, valid loss: 1.607248, train accuracy: 0.969975, valid accuracy: 0.434559
stopping_step = 0
Epoch: 4, train loss: 0.315284, valid loss: 1.592499, train accuracy: 0.996630, valid accuracy: 0.443750
stopping_step = 0
Epoch: 5, train loss: 0.189573, valid loss: 1.588220, train accuracy: 0.998775, valid accuracy: 0.461029
stopping_step = 0
Epoch: 6, train loss: 0.123662, valid loss: 1.588453, train accuracy: 0.999694, valid accuracy: 0.445588
stopping_step = 1
Epoch: 7, train loss: 0.086680, valid loss: 1.592129, train accuracy: 1.000000, valid accuracy: 0.449265
stopping_step = 2
Epoch: 8, train loss: 0.064186, valid loss: 1.595355, train accuracy: 1.000000, valid accuracy: 0.449265
stopping_step = 3
Epoch: 9, train loss: 0.049568, va

In [5]:
np.save(os.path.join('/data/put_data/cclin/ntu/dlcv2018/hw5/results', dir_name, 'models', 'results.npy'), results)

In [6]:
### p1_learning_curve.jpg: learning curve
results = np.load(os.path.join('/data/put_data/cclin/ntu/dlcv2018/hw5/results', dir_name, 'models', 'results.npy'))
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(range(len(results[1])), results[1], label='Validation error')
ax.plot(range(len(results[0])), results[0], label='Training error')
ax.set_xticks(np.arange(len(results[0])))
ax.set_xlabel('Training epochs', fontsize=16)
ax.set_ylabel('Cross entropy', fontsize=16)
ax.set_title('Learning Curve', fontsize=20)
ax.legend(fontsize=16)
fig.savefig(os.path.join('/data/put_data/cclin/ntu/dlcv2018/hw5/results', dir_name, 'p1_learning_curve.jpg'))
plt.close(fig)

In [7]:
tf.reset_default_graph()

with tf.Session() as sess:
    net = DNN(sess,
              model_name=dir_name)
    net.build_model()
    net.inference(feature_path=feature_path_valid,
                  label_file=label_file_valid,
                  gen_from=os.path.join('/data/put_data/cclin/ntu/dlcv2018/hw5/results', dir_name, 'models'),
                  out_path=os.path.join('/data/put_data/cclin/ntu/dlcv2018/hw5/results', dir_name),
                  bsize=bsize)

INFO:tensorflow:Restoring parameters from /data/put_data/cclin/ntu/dlcv2018/hw5/results/cnn_dnn_2048_512_11_bs32_lr5e5_ep50/models/cnn_dnn_2048_512_11_bs32_lr5e5_ep50.model-5
 [*] Success to read cnn_dnn_2048_512_11_bs32_lr5e5_ep50.model-5
 [*] Load SUCCESS
valid loss: 1.588220, valid accuracy: 0.461029


### debug: statistics of number of frames

In [21]:
res = np.load(feature_path_valid)

n_frames_valid = [r.shape[0] for r in res]
print(np.mean(n_frames_valid))
print(np.median(n_frames_valid))
print(np.std(n_frames_valid))

9.367504835589942
5.0
17.749066179769716


In [33]:
sum([i > 30 for i in n_frames_valid]) / len(n_frames_valid)

0.05029013539651837

In [19]:
res = np.load(feature_path_train)

n_frames_train = [r.shape[0] for r in res]
print(np.mean(n_frames_train))
print(np.median(n_frames_train))
print(np.std(n_frames_train))

9.193757725587144
5.0
13.51620272358004


In [34]:
sum([i > 30 for i in n_frames_train]) / len(n_frames_train)

0.061495673671199014